<a href="https://colab.research.google.com/github/Arush0113/ArushTensorflowRepo/blob/main/Customising%20your%20models%20with%20Tensorflow%202/Models_using_Subclassing_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Make the necessary Imports



In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Softmax, Dropout, concatenate

## Define your Model using subclassing API

In [2]:
class MyModel(Model):
  def __init__(self, num_classes, **kwargs):
    super(MyModel, self).__init__(**kwargs)
    self.dense1 = Dense(32, activation='relu')
    self.dense2 = Dense(16)
    self.dense3 = Dense(num_classes)
    self.softmax = Softmax()

  def call(self, inputs):
    h1 = self.dense1(inputs)
    h2 = self.dense2(inputs)
    y = self.dense3(h2)
    concatenated_array = concatenate([h2, y])
    return self.softmax(concatenated_array)

## Initialize Model and check Summary

In [3]:
model1 = MyModel(10, name = 'my_model_1')
model1(tf.random.uniform([1,10]))
model1.summary()

Model: "my_model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                multiple                  352       
_________________________________________________________________
dense_1 (Dense)              multiple                  176       
_________________________________________________________________
dense_2 (Dense)              multiple                  170       
_________________________________________________________________
softmax (Softmax)            multiple                  0         
Total params: 698
Trainable params: 698
Non-trainable params: 0
_________________________________________________________________


## Creating Layers using the subclassing API

In [4]:
from tensorflow.keras.layers import Layer

In [5]:
class MyLayer(Layer):
  def __init__(self, input_dim, units, **kwargs):
    super(MyLayer, self).__init__(**kwargs)
    
    self.w = self.add_weight(
        shape = (input_dim, units),
        initializer = 'random_normal',
        trainable = True
    )
    self.b = self.add_weight(
        shape = (units, ),
        initializer = 'zeros',
        trainable = True
    )

  def call(self, inputs):
    return tf.matmul(inputs, self.w) + self.b

mylayer1 = MyLayer(5, 3)



In [6]:
mylayer1.weights

[<tf.Variable 'Variable:0' shape=(5, 3) dtype=float32, numpy=
 array([[-0.04829543, -0.00438259,  0.12061695],
        [-0.02775233,  0.07999583,  0.01940181],
        [ 0.04043558,  0.02018825,  0.03483267],
        [-0.05835044, -0.01243864,  0.04241115],
        [-0.05231539,  0.02072731, -0.04937466]], dtype=float32)>,
 <tf.Variable 'Variable:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>]

In [7]:
mylayer1(tf.ones((1,5)))

<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[-0.14627802,  0.10409015,  0.16788793]], dtype=float32)>